In [ ]:
import getpass
import os
os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


# Instalação e Importação de Bobliotecas

In [ ]:
!pip install langchain_community langchain_text_splitters langchain_openai faiss-cpu

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Retrieval Augmented Generation (RAG)

## Carregamento dos Documentos (Loading)
Foi utilizado o texto bíblico de Provérbios a partir do [bibliaonline.com](https://www.bibliaonline.com.br/ara)



In [ ]:
# Filtra o conteúdo da página por uma classe específica
bs4_strainer = bs4.SoupStrainer(class_=("FragmentView_text__g6Uq2 FragmentView_verseByVerse__l1TB0"))

docs = []

for i in range(1, 32):
  # Carrega o conteúdo da página
  loader = WebBaseLoader(
      web_paths=(f"https://www.bibliaonline.com.br/ara/pv/{i}",),
      bs_kwargs={"parse_only": bs4_strainer},
  )

  # Carrega o conteúdo da página
  docs.extend(loader.load())

In [ ]:
for i in range(1, 32):
  print(f'Capítulo {i}: {len(docs[i-1].page_content)}')

Capítulo 1: 2841
Capítulo 2: 1780
Capítulo 3: 2867
Capítulo 4: 2236
Capítulo 5: 1970
Capítulo 6: 3065
Capítulo 7: 2139
Capítulo 8: 3149
Capítulo 9: 1453
Capítulo 10: 2742
Capítulo 11: 2652
Capítulo 12: 2496
Capítulo 13: 2170
Capítulo 14: 3004
Capítulo 15: 2897
Capítulo 16: 2850
Capítulo 17: 2484
Capítulo 18: 1941
Capítulo 19: 2561
Capítulo 20: 2556
Capítulo 21: 2525
Capítulo 22: 2461
Capítulo 23: 2924
Capítulo 24: 3054
Capítulo 25: 2538
Capítulo 26: 2433
Capítulo 27: 2394
Capítulo 28: 2699
Capítulo 29: 2332
Capítulo 30: 3185
Capítulo 31: 2392


## Dividindo Documentos (Splitting/Chunking)

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["Capítulo ", "Versículo "]
)

all_splits = text_splitter.split_documents(docs)

len(all_splits)

31

## Indexando (Store)

In [ ]:
vectorstore = FAISS.from_documents(all_splits, OpenAIEmbeddings())

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("Como ser mais sábio?")

len(retrieved_docs)

6

In [ ]:
system_template = """Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.

Pergunta: {question}

Contexto: {context}

Resposta:
"""

prompt_template = ChatPromptTemplate.from_template(system_template)

In [ ]:
example_messages = prompt_template.invoke({
    "context": "algum contexto",
    "question": "alguma pergunta"
})

print(example_messages.to_messages())

[HumanMessage(content='Você é um assistente para tarefas de perguntas e respostas. Use os seguintes trechos de contexto recuperados para responder à pergunta. Se você não souber a resposta, apenas diga que não sabe. Use no máximo duas frases e mantenha a resposta concisa e fale apenas o necessário.\n\nPergunta: alguma pergunta\n\nContexto: algum contexto\n\nResposta:\n')]


## Gerando Respostas (Generate)

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt_template
    | llm
    | StrOutputParser()
)

In [ ]:
for chunk in rag_chain.stream("O que o livro de Provérbios fala sobre ovelhas?"):
    print(chunk, end="", flush=True)

O livro de Provérbios menciona a importância de cuidar das ovelhas e dos rebanhos, simbolizando a responsabilidade e a diligência no manejo dos bens. Além disso, destaca que as riquezas não duram para sempre, enfatizando a necessidade de sabedoria na administração.